In [1]:
import numpy as np
import plotly.express as px
import pandas as pd
import plotly.graph_objects as go
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp

## Single Mode

In [2]:
# Print Out Readings Table
readings = pd.DataFrame()
Least_Count_Main_Scale = 2 # Degree
Least_Count_Vernier_Scale = 0.2
readings['Angle (degree)'] =  np.array([ 70.4, 74.6, 75.6, 76.6, 77.0, 77.2, 77.4, 78.4, 78.8, 79.0, 80.0, 80.6, 80.8, 81.0, 81.6, 81.8, 82.0, 82.6, 82.8, 83.0, 83.2,
                               83.4, 83.6, 84.0, 84.2, 84.6, 85.6, 86.4, 87.0, 87.4, 88.0, 88.6, 90.4, 90.6 ,90.8, 92.8, 93.6, 94.0 ]) 
readings['Scaled Angle (degree)'] = readings['Angle (degree)'] - 82.0
error_in_current = 0.8 # Zero Error
readings['Current (micro-Amp)'] =  np.array([ 1, 9.9, 19.6, 42, 56, 71, 84, 100, 200, 300, 600, 900, 1000, 1200, 1400, 1500, 1600, 1500, 1400, 1300,
                                             1200, 1100, 1000, 900, 800, 600, 300, 100, 71, 50, 40, 20, 10, 5.2, 3.6, 1.5, 1.0, 0.9 ]) - error_in_current

readings['Log Current'] = np.log(readings['Current (micro-Amp)'])
readings

,Angle (degree),Scaled Angle (degree),Current (micro-Amp),Log Current
0,70.4,-11.6,0.2,-1.609438
1,74.6,-7.4,9.1,2.208274
2,75.6,-6.4,18.8,2.933857
3,76.6,-5.4,41.2,3.718438
4,77.0,-5.0,55.2,4.010963
5,77.2,-4.8,70.2,4.251348
6,77.4,-4.6,83.2,4.421247
7,78.4,-3.6,99.2,4.597138
8,78.8,-3.2,199.2,5.294309
9,79.0,-3.0,299.2,5.701112


In [3]:
readings.to_csv('Readings_Exp2_Fiber_Optics.csv')

***

# Curve Fitting using 2nd Order Quadratic Eqn

In [4]:
x = readings['Scaled Angle (degree)'].values
# Taking Log of Current 
y = np.log(readings['Current (micro-Amp)'])

In [5]:
# Quadratic Equation Function
def Quad(X,C,w):
    return -w*(X**2)+C  # w and C ----> fitting parameters

In [6]:
# Applying fitting algorithm
popt,pcov = curve_fit(Quad,x,y)
popt,pcov # printing fitted parameters

(array([6.56919874, 0.06532832]), array([[1.83126127e-02, 2.15787538e-04],
        [2.15787538e-04, 6.97154095e-06]]))

In [7]:
# Calculating Current using fitted quadratic equation
xx = np.linspace(x.min(),x.max(),num=2000)
yy = Quad( xx, C=popt[0], w=popt[1])

# Plotting Observed value and fitted values
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=xx, y=yy, mode='lines', name='Fiited'))
fig.update_layout( height=750, width=1500,  xaxis_title='Scaled Angle (degree)', yaxis_title='Log of Current',
                  title="Log of Current (micro-Amp) vs Scaled Angle (degree) | Single Mode | 2nd Order Fitting ")

In [8]:
# Taking exp() of fitted values of log(Current) that is Fitted Current
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.exp(y), mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=xx, y=np.exp(yy), mode='lines', name='Fitted Curve'))

# Curve fitted using 10th order Quadratic Equation

In [9]:
x = readings['Scaled Angle (degree)'].values
y = np.log(readings['Current (micro-Amp)']) # Taking Log of current

# Applying Curve fitting Algorithm
fitted_polynomial_coeff = np.polyfit(x,y,10)
fitted_polynomial = np.poly1d(fitted_polynomial_coeff)

# Calculating fitted values using fitted curve
fitted_data = pd.DataFrame()
fitted_data['Angle (degree) Fitted'] = np.linspace(readings['Scaled Angle (degree)'].min()-0.0018,readings['Scaled Angle (degree)'].max(),num=2000)
fitted_data['Current (micro-Amp) Fitted'] = fitted_polynomial(fitted_data['Angle (degree) Fitted'])

# Create Graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=fitted_data['Current (micro-Amp) Fitted'], mode='lines', name='Fitted Curve'))
fig.update_layout( height=650, width=1400,  xaxis_title='Scaled Angle (degree)', yaxis_title='Log of Current',
                  title="Log of Current vs Scaled Angle (degree) | Single Mode | 10th Order fitting ")

In [10]:
# Taking exp() of fitted values of log(Current) that is Fitted Current
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.exp(y), mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=np.exp(fitted_data['Current (micro-Amp) Fitted']), mode='lines', name='Fitted Curve'))
fig.update_layout( height=650, width=1400,  xaxis_title='Scaled Angle (degree)', yaxis_title='Current',
                  title="Current vs Scaled Angle (degree) | Single Mode | 10th Order fitting ")

# Scaling the Current Values as well 
and then applying curve fitting

In [11]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [13]:
# Applying Scaling Algorithm
scaler.fit( readings['Current (micro-Amp)'].values.reshape(-1,1) )
current_scaled = scaler.transform( readings['Current (micro-Amp)'].values.reshape(-1,1) )

x = readings['Scaled Angle (degree)'].values
# Taking Log of Scaled Current
y = np.log(current_scaled)

# Applying Curve Fitting Algorithm
fitted_polynomial_coeff = np.polyfit(x,y,2)
fitted_polynomial = np.poly1d(fitted_polynomial_coeff)

# Calculating Fitted Values of Log of Current
fitted_data = pd.DataFrame()
fitted_data['Angle (degree) Fitted'] = np.linspace(readings['Scaled Angle (degree)'].min()-0.0018,readings['Scaled Angle (degree)'].max(),num=2000)
fitted_data['Current (micro-Amp) Fitted'] = fitted_polynomial(fitted_data['Angle (degree) Fitted'])

# Create traces
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=fitted_data['Current (micro-Amp) Fitted'], mode='lines', name='Fitted Curve'))

fig.update_layout( height=650, width=1400,  xaxis_title='Scaled Angle (degree)', yaxis_title='Log of Current',
                  title="Log of Current vs Scaled Angle (degree) | Single Mode | 10th Order fitting ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning:

divide by zero encountered in log



ValueError: ignored

Error : Cannot take log of current after scaling it because after scaling some zero values are there

# Gaussian Curve Fitting

In [14]:
fitted_data = pd.DataFrame()
fitted_data['Angle (degree) Fitted'] = np.linspace(readings['Scaled Angle (degree)'].min(),readings['Scaled Angle (degree)'].max(),num=2000)

#Calculating the Gaussian PDF values given Gaussian parameters and random variable X
def gaus(X,C,X_mean,sigma): 
    return C*np.exp(-(X-X_mean)**2/(2*sigma**2)) # C, X_mean and sigma are the fitting parameters

x = fitted_data['Angle (degree) Fitted']
y = readings['Current (micro-Amp)']
# mean = sum(x*y)/sum(y)                  
# sigma = sum(y*(x-mean)**2)/sum(y) 

# Fitted Parameters
fitted_data['Current (micro-Amp) Fitted'] = gaus( fitted_data['Angle (degree) Fitted'], C=1575.29637, X_mean= 0.214060, sigma = 1.556317  )

# Create Graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=readings['Scaled Angle (degree)'], y=readings['Current (micro-Amp)'], mode='markers', name='Observations', marker=dict(size=8 )))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=fitted_data['Current (micro-Amp) Fitted'], mode='lines', name='Fitted Curve'))

Max_Point_on_y = fitted_data['Current (micro-Amp) Fitted'].max()
y_cut = Max_Point_on_y*(1/(np.exp(2)))

# Marking Points and Lines on Graphs
fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )

x_1 = 0.214060 - 2*1.556317
x_2 = 0.214060 + 2*1.556317

fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )
fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )

fig.add_shape(type="line", x0=x_1, y0=0, x1=x_1, y1=y_cut )
fig.add_shape(type="line", x0=x_2, y0=0, x1=x_2, y1=y_cut )

fig.add_annotation(  x=x_1, y=y_cut, text="("+str(np.round(x_1,3))+","+str(np.round(y_cut,3))+")", arrowhead=2, ax=-0.1 )
fig.add_annotation(  x=x_2, y=y_cut, text="("+str(np.round(x_2,3))+","+str(np.round(y_cut,3))+")", arrowhead=2 )

fig.update_layout( height=950, width=1750,  xaxis_title='Scaled Angle (degree)', yaxis_title='Current (micro-Amp)',
                  title="Current (micro-Amp) vs Scaled Angle (degree) | Single Mode ",
                  legend=dict( yanchor="top", y=0.99, xanchor="left",  x=0.01),
                  font=dict(size=15))
fig.show()

***

***

# Using tan(theta)

# Quardratic Curve Fitting

In [15]:
x = np.tan(readings['Angle (degree)'].values)
y = np.log(readings['Current (micro-Amp)']) # Taking Log of current

# Applying Curve fitting Algorithm
fitted_polynomial_coeff = np.polyfit(x,y,2)
fitted_polynomial = np.poly1d(fitted_polynomial_coeff)

# Calculating fitted values using fitted parameters
fitted_data = pd.DataFrame()
fitted_data['Angle (degree) Fitted'] = np.linspace(x.min(),x.max(),num=2000)
fitted_data['Current (micro-Amp) Fitted'] = fitted_polynomial(fitted_data['Angle (degree) Fitted'])

# Create Graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=y, mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=fitted_data['Current (micro-Amp) Fitted'], mode='lines', name='Fitted Curve'))
fig.update_layout( height=650, width=1400,  xaxis_title='tan of angle (degree)', yaxis_title='Log of Current',
                  title="Log of Current vs tan of angle (degree) | Single Mode | 2nd Order fitting ")

In [16]:
# Taking exp() of fitted values of log(Current) that is Fitted Current
fig = go.Figure()
fig.add_trace(go.Scatter(x=x, y=np.exp(y), mode='markers', name='Observations'))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=np.exp(fitted_data['Current (micro-Amp) Fitted']), mode='lines', name='Fitted Curve'))
fig.update_layout( height=650, width=1400,  xaxis_title='Scaled Angle (degree)', yaxis_title='Current',
                  title="Current vs Scaled Angle (degree) | Single Mode | 10th Order fitting ")

# Gaussian Curve Fitting

In [17]:
#Calculating the Gaussian PDF values given Gaussian parameters and random variable X
def gaus(X,C,B,X_mean): 
    return B*np.exp(-C*((X-X_mean)**2)) # C, X_mean and B are the fitting parameters

x = np.tan( readings['Angle (degree)'].values )
y = readings['Current (micro-Amp)'].values 

In [18]:
# Fitted Parameters
popt,pcov = curve_fit(gaus,x,y, p0=[1,1,0])
popt,pcov

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: RuntimeWarning:

overflow encountered in exp



(array([-1.83601347e-04,  5.44394175e+01, -1.10748248e+02]),
 array([[ 7.02900872e-08, -3.28160057e-02, -3.62995456e-02],
        [-3.28160057e-02,  3.11321347e+04,  2.38818210e+04],
        [-3.62995456e-02,  2.38818210e+04,  2.18041648e+04]]))

In [19]:
# fitted_data['Current (micro-Amp) Fitted'] = gaus( fitted_data['Angle (degree) Fitted'], C=1575.29637, X_mean= 0.214060, sigma = 1.556317  )

# Create Graph
fig = go.Figure()
fig.add_trace(go.Scatter(x=readings['Scaled Angle (degree)'], y=readings['Current (micro-Amp)'], mode='markers', name='Observations', marker=dict(size=8 )))
fig.add_trace(go.Scatter(x=fitted_data['Angle (degree) Fitted'], y=fitted_data['Current (micro-Amp) Fitted'], mode='lines', name='Fitted Curve'))

Max_Point_on_y = fitted_data['Current (micro-Amp) Fitted'].max()
y_cut = Max_Point_on_y*(1/(np.exp(2)))

# Marking Points and Lines on Graphs
fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )

x_1 = 0.214060 - 2*1.556317
x_2 = 0.214060 + 2*1.556317

fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )
fig.add_shape(type="line", x0=-11.6, y0=y_cut, x1=12, y1=y_cut )

fig.add_shape(type="line", x0=x_1, y0=0, x1=x_1, y1=y_cut )
fig.add_shape(type="line", x0=x_2, y0=0, x1=x_2, y1=y_cut )

fig.add_annotation(  x=x_1, y=y_cut, text="("+str(np.round(x_1,3))+","+str(np.round(y_cut,3))+")", arrowhead=2, ax=-0.1 )
fig.add_annotation(  x=x_2, y=y_cut, text="("+str(np.round(x_2,3))+","+str(np.round(y_cut,3))+")", arrowhead=2 )

fig.update_layout( height=950, width=1750,  xaxis_title='Scaled Angle (degree)', yaxis_title='Current (micro-Amp)',
                  title="Current (micro-Amp) vs Scaled Angle (degree) | Single Mode ",
                  legend=dict( yanchor="top", y=0.99, xanchor="left",  x=0.01),
                  font=dict(size=15))
fig.show()